In [1]:
# Run cell to get the Spark session as spark, use config variable to check the configurations.
import os;os.environ['executor_pod_image'] = os.getenv('DOCKER_REGISTRY', 'registry.lti-aiq.in:443') + '/mosaic-ai-logistics/mosaic-notebooks-manager/mosaic-docker-build/spark-executor:1.0.2'

from spark_distributed.spark_distributed_session import spark, config
spark

/opt/conda/lib/python3.7/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Launching Spark Session!



22/07/24 16:14:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
ls -al /data

total 58980
drwxrwxrwx. 2 mosaic-ai mosaic-ai     4096 Jul 24 16:12  ./
drwxr-xr-x. 1 root      root           103 Jul 24 15:56  ../
-rwxrwxr-x. 1 jovyan         1000 59589785 Jul 24 15:53  faker_data_1lakh48col.csv*
-rwxr-xr-x. 1 jovyan         1000   104834 Jul 24 15:53  sample_libsvm_data.txt*
-rwxrwxrwx. 1 mosaic-ai mosaic-ai   687732 Jul 24 15:52 'Spark Modal Registration and Deployment Testing.html'*


In [3]:
from __future__ import print_function
from mosaicml import *
from mosaicml.constants import MLModelFlavours
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
import numpy

In [4]:
data = spark.read.format("libsvm").load("/data/sample_libsvm_data.txt")

22/07/24 16:14:49 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [5]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
splits = data.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

In [7]:
model = nb.fit(train)

predictions = model.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+-----+--------------------+--------------------+-----------+----------+
|label|            features|       rawPrediction|probability|prediction|
+-----+--------------------+--------------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|[-172664.79564650...|  [1.0,0.0]|       0.0|
|  0.0|(692,[98,99,100,1...|[-176279.15054306...|  [1.0,0.0]|       0.0|
|  0.0|(692,[122,123,124...|[-189600.55409526...|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[-274673.88337431...|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[-183393.03869049...|  [1.0,0.0]|       0.0|
|  0.0|(692,[125,126,127...|[-256992.48807619...|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[-210411.53649773...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-170627.63616681...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-212157.96750469...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-183253.80108550...|  [1.0,0.0]|       0.0|
|  0.0|(692,[128,129,130...|[-246528.93739632...|  

In [8]:
from werkzeug.datastructures import FileStorage
@scoring_func
def score(model, request):
    file= FileStorage(request.files['file1'])
    file.save("/data/f1.txt")
    spark_new = SparkSession.builder.appName("NaiveBayesExample").getOrCreate()
    data = spark_new.read.format("libsvm") \
        .load("/data/f1.txt")
#   Split the data into train and test
    splits = data.randomSplit([0.6, 0.4], 1234)
    train = splits[0]
    test = splits[1]
    
    predictions = model.transform(test)
    print("values1", predictions)
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                                  metricName="accuracy")
    print(predictions.show())
    accuracy = evaluator.evaluate(predictions)
    print("Test set accuracy = " + str(accuracy))
    return accuracy

In [9]:
import json
import os
import shutil
import tempfile
import uuid
import time
from json import JSONDecodeError
import platform
import PIL
import requests
from datetime import datetime
from mosaic_utils.ai.build_time_metrics.metrics import metrics_stats
from mosaic_utils.ai.encoding_utils import base64_encode
from mosaic_utils.ai.file_utils import (
    create_model_tar,
    extract_tar,
    pickle_dumps,
    pickle_loads,
)
from requests_toolbelt.multipart.encoder import MultipartEncoder
from mosaic_utils.ai.validate.validators import ValidationHandler
from mosaicml.widgets.registered_output import ModelDescribe
import logging as log

def load_model(ml_model_id, version_id):
    """
    Method to construct the model object from the model and version id

    Args:
        ml_model_id (string): identifier for ml_model
        version_id (string): identifier for version_id

    Returns:
        tuple
    """
    # fetch model metadata
    print("Inside Load_modal")
    model_info = describe_model(ml_model_id)
    # download model artifacts
    url = MosaicAI.server + MLModelVersionV1.download.format(
        ml_model_id=ml_model_id, version_id=version_id
    )
    print(f"Downloading content form url: {url}")
    response = requests.get(url, headers=get_headers())
    print(f"response: {response.content}")
    # create temperory directory
    base_dir = "/data/tmp/"
    # write downloaded artifacts to tar file
    tar_path = os.path.join(base_dir, "ml_model.tar.gz")
    print(f"tar_path: {tar_path}")
    tar_file = open(tar_path, "wb")
    print("file opened")
    tar_file.write(response.content)
    tar_file.close()
    print("file closed")
    # extract tar file
    extract_tar(tar_path, base_dir)
    print("After extract_tar")
    # build model object
    flavour = model_info["flavour"]
    # build scoring function
    scoring_func_path = os.path.join(base_dir, "scoring_func")
    scoring_func = pickle_loads(scoring_func_path)
    if flavour == MLModelFlavours.ensemble:
        return scoring_func, model_info["versions"][0]["dependent_model"]

    model_handler = get_flavour_handler(flavour)
    print(f"model_handler: {model_handler}")
    model_path = os.path.join(base_dir, "ml_model")
    print(f"model_path: {model_path}")
    model = model_handler.load_model(model_path)
    # delete temperory directory
    shutil.rmtree(base_dir)
    return model, scoring_func

class MosaicAI:
    server = os.getenv("MOSAIC_AI_SERVER", "http://localhost:5000/registry/api")
    kyd_setting = "/v1/ml-model/{}/version/{}/kyd/settings"
    
class MLModelVersionV1:
    lc = "/v1/ml-model/{ml_model_id}/version"
    rud = "/v1/ml-model/{ml_model_id}/version/{version_id}"
    upload = "/v1/ml-model/{ml_model_id}/version/{version_id}/upload"
    download = "/v1/ml-model/{ml_model_id}/version/{version_id}/download"
    build_time_metrics = "/v1/ml-model/metrics/{version_id}/{tag}"
    delete = "/v1/ml-model/{ml_model_id}/version/{version_id}"

def get_headers():
    jwt = read_jwt()
    return {
        Headers.authorization: f"Token {jwt}",
        Headers.x_project_id: os.environ.get("PROJECT_ID"),
    }

def read_jwt():
    config_file = os.path.expanduser(Client.config_file)
    if os.path.exists(config_file):
        with open(config_file) as f:
            jwt = f.read().strip("\n")
        return jwt
    elif os.getenv("TOKEN"):
        jwt = os.getenv("TOKEN")
        return jwt
    raise ConfigError

class Client:
    config_file = "~/.mosaic.ai"

class Headers:
    authorization = "Authorization"
    x_project_id = "X-Project-Id"
    
def get_model_type(model_path):
    """Get type of pyspark model."""
    print(f"model_path: {model_path}")
    metadata_path = os.path.join(model_path, "metadata")
    metadata_files = os.listdir(metadata_path)
    metadata_file = [x for x in metadata_files if x.startswith("part-000")]
    if not metadata_file:
        raise FileNotFoundError("Model metadata file not found")
    metadata_file = metadata_file[0]
    with open(os.path.join(metadata_path, metadata_file), "r") as f_handle:
        model_metadata = f_handle.read()
    model_metadata = json.loads(model_metadata)
    model_class = model_metadata.get("class")
    if not model_class:
        raise AttributeError('Attribute "class" not found in model metadata')
    model_class = model_class.replace("org.apache.spark", "pyspark")
    return model_class

In [10]:
# Register Model

from mosaic_utils.ai.validate.validators import ValidationHandler
from mosaicml.widgets.registered_output import ModelDescribe
import logging as log
from mosaic_utils.ai.flavours import keras, pyspark, pytorch, sklearn, spacy, tensorflow

class MLModelV1:
    c = "/v1/ml-model/register"
    u = "/v1/ml-model/add-version"
    lc = "/v1/ml-model"
    rud = "/v1/ml-model/{ml_model_id}"
    rud_model_name = "/v1/ml-model/{ml_model_id}?ml_model_name={ml_model_name}"
    rud_model_name1 = "/v1/ml-model/{ml_model_id}?ml_model_name={ml_model_name}&type={model_type}"

    
def create_artifacts(
    flavour,
    model_obj,
    scoring_func_from_user,
    x_train=None,
    model_list=None,
    x_test=None,
    y_train=None,
    y_test=None,
    custom_score=None,
):
    # create temporary directory
    temp_dir = "/data/tmp/"
    # serialize model
    model_file = _pickle_model(flavour, model_obj, temp_dir)
    # serialize scoring function
    scoring_func_file = _pickle_scoring_func(scoring_func_from_user, temp_dir)
    if model_list:
        model_list_file = _pickle_model_list(model_list, temp_dir)
        return (
            temp_dir,
            create_model_tar(
                temp_dir, None, model_file, scoring_func_file, model_list_file
            ),
        )
    print(f"temp_dir: {temp_dir}\nmodel_file: {model_file}\nscoring_func_file:{scoring_func_file}")
    # generate tar file
    tar_file = create_model_tar(
            temp_dir,
            None,
            model_file,
            scoring_func_file,
            _pickle_training_test_data(x_train, temp_dir, "x_train"),
            _pickle_training_test_data(x_test, temp_dir, "x_test"),
            _pickle_training_test_data(y_train, temp_dir, "y_train"),
            _pickle_training_test_data(y_test, temp_dir, "y_test"),
            _pickle_training_test_data(custom_score, temp_dir, "custom_score"),
        )
    # delete from location once tar is created
    print(f"Model Flavor: {flavour}")
#     if flavour=="pyspark" and os.path.exists(model_file):
#         print(f"Removing model file - {model_file}")
#         shutil.rmtree(model_file)
    print(f"temp_dir: {temp_dir}\n\ntar_file:{tar_file}")
    return (
        temp_dir,
        tar_file,
    )


def register_model(
    model_obj,
    scoring_func,
    name,
    description,
    flavour,
    tags=None,
    init_script=None,
    schema=None,
    y_true=None,
    y_pred=None,
    prob=None,
    features=None,
    labels=None,
    model_type=None,
    datasource_name=None,
    metadata_info=None,
    input_type="json",
    target_names=None,
    target_names_mapping=None,
    x_train=None,
    y_train=None,
    feature_names=None,
    feature_ids=None,
    explain_ai=False,
    x_test=None,
    y_test=None,
    kyd=False,
    kyd_score=False,
    pretty_output=True,
    custom_score=None,
    **kwargs
):
    validation_handler = ValidationHandler(
        mandatory_fields={"name": name,
                          "description": description,
                          "flavour": flavour,
                          "scoring_func": scoring_func,
                          },
        alphanum_uscore_excl={"name": name
                              },
        if_present_validate_type={'schema': (schema, (dict, )),
                                  'metadata_info': (metadata_info, (dict, )),
                                  'tags': (tags, (list, ))
                                  },
        if_present_sub_field_must_exist={'kyd': (kyd, {"x_train": x_train,
                                                       "y_train": y_train,
                                                       "x_test": x_test,
                                                       "y_test": y_test,
                                                       "feature_names": feature_names,
                                                       "model_type": model_type,
                                                       "kyd_score": kyd_score}),
                                         },
    )
    validation_status_, message_ = validation_handler.validate()
    if validation_status_ is False:
        return message_



    global kyd_executor
    # create model and version
    if init_script is None:
        init_script = ""
    if model_type:
        metrics_parameter_validation(model_type, y_true, y_pred, prob)
    if tags is not None:
        tags = list(set(tags))
        tags = {"user_tags": tags}
        tags = json.dumps(tags)

    # generate model artifacts
    artifacts_dir, artifacts_tar = create_artifacts(
    flavour,
    model_obj,
    scoring_func,
    x_train=x_train,
    y_train=y_train,
    x_test=x_test,
    y_test=y_test,
    custom_score=custom_score,
    )
    print(f"artifacts_dir: {artifacts_dir}\n\nartifacts_tar: {artifacts_tar}")

    type_inference = {}
    model_info = {
        "mode": model_type,
        "targets_mapping": dict(
            target_names=target_names, mapping_value=target_names_mapping
        ),
        "features_name": kwargs.get("explicit_feature_names")
        if kwargs.get("explicit_feature_names", None)
        else feature_names,
        "features_type": type_inference,
        "feature_type_inferenced": bool(type_inference),
        "number_of_features": len(feature_names) if feature_names else 0,
        "number_of_targets": len(target_names) if target_names else 0,
        "deep_learning_model": False,
        "temp_dir": "",
        "expai": explain_ai,
        "kyd": kyd,
    }
    source = kwargs.get("source") if kwargs.get("source") else None
    model_display = kwargs.get("model_display")
    if model_display is None:
        model_display = True

    # register model
    ml_model = register(
        name,
        description,
        flavour,
        schema,
        metadata_info,
        model_display=model_display,
        source=source,
        init_script=create_init_script(init_script, flavour),
        input_type=input_type,
        target_names={"target": target_names},
        datasource_name=datasource_name,
        model_class=get_model_structure(model_obj, flavour),
        tar_file=artifacts_tar,
        tags=tags,
        model_info=model_info,
        score_file=dump_score_func(scoring_func)
    ) 
    _ml_model_id = ml_model["id"]
    ml_model["versions"].sort(key=lambda x: x["created_on"])
    _ml_version_id = ml_model["versions"][-1]["id"]
    model_summary = describe_model(_ml_model_id)
    detailed_matrix = {}
    # calculate model metrics
    if model_type:
        metrics = metrics_stats(
            None,
            _ml_model_id,
            _ml_version_id,
            y_true,
            y_pred,
            prob,
            model_type,
            model_summary,
            model_obj,
            labels,
            features,
            feature_names,
            None,
        )
        detailed_matrix = metrics.detailed_matrix()
    project_id = ml_model["project_id"]
    store_model_profiling(datasource_name, _ml_version_id, _ml_model_id, project_id)

    if pretty_output:
        described_model = ModelDescribe(_ml_model_id)
        described_model.view()
    else:
        described_model = describe_model(_ml_model_id)
        described_model["versions"][-1]["detailed_matrix"] = detailed_matrix
        return described_model


def register(
    name,
    description,
    flavour,
    schema,
    metadata_info,
    init_script,
    input_type,
    target_names,
    datasource_name,
    model_class,
    tar_file,
    model_display=None,
    source=None,
    tags=None,
    model_info=None,
    model_list=None,
    score_file=None
):
    if model_list:
        model_list = [
            {
                k: v
                for k, v in list_meta.items()
                if k
                not in [
                    "flavour",
                    "init_script",
                    "description",
                    "previous_model_id",
                    "previous_version_id",
                ]
            }
            for list_meta in model_list
        ]
    if type(score_file) == str:
        score_file_data = ("score_func.pkl", open(score_file, "rb"))
    else:
        score_file_data = None

    print(f"metadata_info: {metadata_info}")
    multipart_data = MultipartEncoder(
        fields={
            "tar_file": ("ml_model.tar.gz", open(tar_file, "rb")),
            "score_file": score_file_data,
            "name": name,
            "description": description,
            "flavour": flavour,
            "tags": tags,
            "init_script": base64_encode(init_script),
            "input_type": input_type,
            "target_names": json.dumps(target_names),
            "datasource_name": datasource_name,
            "model_class": model_class,
            "schema": json.dumps(schema),
            "metadata_info": json.dumps(metadata_info),
            "model_info": json.dumps(model_info),
            "model_display": json.dumps(model_display),
            "source": source,
            "model_list": json.dumps(model_list),
            "base_id": platform.python_version(),
        }
    )
    url = MosaicAI.server + MLModelV1.c
    headers = get_headers()
    headers["Content-Type"] = multipart_data.content_type
    response = requests.post(url, data=multipart_data, headers=headers)
    if response.status_code != 200:
        raise Exception(response.text)
    return response.json()


def _pickle_model(flavour, model_obj, base_dir):
    model_path = os.path.join(base_dir, "ml_model")
    if flavour == MLModelFlavours.ensemble:
        pickle_dumps(model_obj, model_path)
    else:
        model_handler = get_flavour_handler(flavour)
        model_handler.dump_model(model_obj, model_path)
    return model_path

def get_flavour_handler(flavour):
    if flavour == MLModelFlavours.sklearn:
        return sklearn
    if flavour == MLModelFlavours.pytorch:
        return pytorch
    if flavour == MLModelFlavours.keras:
        return keras
    if flavour == MLModelFlavours.tensorflow:
        return tensorflow
    if flavour == MLModelFlavours.pyspark:
        return pyspark
    if flavour == MLModelFlavours.spacy:
        return spacy
    
def _pickle_scoring_func(scoring_func_from_user, base_dir):
    scoring_func_path = os.path.join(base_dir, "scoring_func")
    pickle_dumps(scoring_func_from_user, scoring_func_path)
    return scoring_func_path

def _pickle_training_test_data(data, base_dir, data_type):
    """
    This function is used to pickle our training and test data set
    :param data:
    :param base_dir:
    :param data_type:
    :return:
    """
    data_path = os.path.join(base_dir, data_type)
    pickle_dumps(data, data_path)
    return data_path

def create_init_script(user_init_script, flavour):
    if flavour == MLModelFlavours.r:
        print("inside init script")
        # fetched_pip_packages = generate_init_script()
        fetched_pip_packages = ""
        r_installation = create_r_installation(fetched_pip_packages)
        init_script = r_installation + "\\n" + user_init_script + '"'
        return init_script
    
    init_script = user_init_script
    return init_script


def get_model_structure(model_obj, flavour):
    try:
        model_structure = get_flavour_handler(flavour).get_model_structure(model_obj)
    except:
        model_structure = json.dumps({"class": str(model_obj.__class__)[8:-2:]})
    return model_structure

def dump_score_func(score_func):
    temp_dir = tempfile.mkdtemp()
    scoring_func_path = os.path.join(temp_dir, "score_func.pkl")
    pickle_dumps(score_func, scoring_func_path)
    return scoring_func_path

def store_model_profiling(datasource_name, ml_version_id, ml_model_id, project_id):
    if datasource_name:
        url = MosaicAI.server + MLModelProfiling.c
        payload = {
            "datasource_name": datasource_name,
            "version_id": ml_version_id,
            "ml_model_id": ml_model_id,
            "project_id": project_id,
        }
        response = requests.post(url, json=payload, headers=get_headers())
        return response.status_code

In [11]:
register_model(model, score, "sample_libsvm_pyspark14", "pyspark model", MLModelFlavours.pyspark)

temp_dir: /data/tmp/
model_file: /data/tmp/ml_model
scoring_func_file:/data/tmp/scoring_func
Model Flavor: pyspark
temp_dir: /data/tmp/

tar_file:/data/tmp/ml_model.tar.gz
artifacts_dir: /data/tmp/

artifacts_tar: /data/tmp/ml_model.tar.gz
metadata_info: None


In [12]:
load_model("2a0e8c72-f05d-4616-bcfa-4e5211b340c5", "f9b8fa99-3ea2-4df8-addd-c3170e771521")

Inside Load_modal
response: b'\x1f\x8b\x08\x08\x0bp\xddb\x02\xffml_model.tar\x00\xed|\x07\\S\xc9\xd6x\x12\x12\xaa\xf4\xdc\x9bB\x08\xa1\x8a\n!tAT\x10A\x94"\xcd\x86"\x04\x12 \x10HL\x02\x02\x82\xa2bAA\x05Q\x11Q\xb1\x80}\xed\x88\xfd\xd8\xbbb/\xab+\xea\xae\x8ak\x17\xcb*\xea\x7f\x12DQ\xf7\xad\xbb\xdf\xdb\xf5\xbd\xef\xff\xe5\xc8\xcf\xdc\x999\xe7\xcc\x999\xf5\xce\xcdM\xba8.]"\x10\x8a\x9d\x08\xff\x1c\xf0\x10x\xba\xbb+?\x9d=\xdd\x9d\xdb\x7f\xb6\x01\xc1\xd9\xcd\xc5\xc3\x13\xfdy\xba\xf1\x08<g77gw\x02\xc7\x9d\xf0\x1d S\xae\xe0\xcb8\x1cB\xbaD\xce\x17%:\xf2E\xff\x02\xef[\xe3m\x0bi\xfb\xfc_\x02\xe9m\xfa\x17\xf0\x15|\xa7\xff\x06\xfd\xa3~gwW\x17O\xb5\xfe\xbf\xbf\xfe\xb9qQ\x03\xfc\xfd\x03\xa2\xa2\xb8\x89\xb2\xc4\xbfU\xff\x1enn\xffZ\xff\xce_\xea\xdf\xd3\xd5\xd5\x8d\xc0\xe1\xa9\xf5\xff\x8fC\xab\x9eI\x045\xfc\xdf\x84/\xfc_\xca\x97)\x1cU\x06\xec\xc8\xf3H\xe2%\xf2\x12\x04\x8e]\xf9I\x1e\x8enIn^\x8e|\x81\xb3\x97\xa3;\xcfK\xe0\xcewv\xf7r\xf7\xf4rLD\x98\\y\x06_*\xcdQ\xd2\x8e\xc8\x14*\xbe\x0e\x1d\xdf\xf2\x7f\x94\x

(NaiveBayesModel: uid=NaiveBayes_b2c6647a2f29, modelType=multinomial, numClasses=2, numFeatures=692,
 <mosaic_utils.ai.decorators.scoring_func at 0x7f1270d64350>)

In [13]:
!ls -al /data

total 58980
drwxrwxrwx. 2 mosaic-ai mosaic-ai     4096 Jul 24 16:16  .
drwxr-xr-x. 1 root      root           103 Jul 24 15:56  ..
-rwxrwxr-x. 1 jovyan         1000 59589785 Jul 24 15:53  faker_data_1lakh48col.csv
-rwxr-xr-x. 1 jovyan         1000   104834 Jul 24 15:53  sample_libsvm_data.txt
-rwxrwxrwx. 1 mosaic-ai mosaic-ai   687732 Jul 24 15:52 'Spark Modal Registration and Deployment Testing.html'


In [14]:
import requests
from werkzeug.datastructures import FileStorage
request = requests.Request()
request.files= {"file1" : open('/data/sample_libsvm_data.txt','rb')}
score(model,request)

22/07/24 16:17:12 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


values1 DataFrame[label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]
+-----+--------------------+--------------------+-----------+----------+
|label|            features|       rawPrediction|probability|prediction|
+-----+--------------------+--------------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|[-172664.79564650...|  [1.0,0.0]|       0.0|
|  0.0|(692,[98,99,100,1...|[-176279.15054306...|  [1.0,0.0]|       0.0|
|  0.0|(692,[122,123,124...|[-189600.55409526...|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[-274673.88337431...|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[-183393.03869049...|  [1.0,0.0]|       0.0|
|  0.0|(692,[125,126,127...|[-256992.48807619...|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[-210411.53649773...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-170627.63616681...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-212157.96750469...|  [1.0,0.0]|       0.0|
|  0.0|(6

1.0

In [ ]:
spark.stop()